#Implementasi CNN untuk Data MNIST

##A. Import Librari yang dibutuhkan:

1. Tensorflow
2. cv2_imshow, sebagai pengganti cv2.imshow() di colab
3. Numpy

In [ ]:
#import librari yang dibutuhkan
import tensorflow as tf
#gunakan cv2_imshow di colab sebagai pengganti cv2.imshow()
from google.colab.patches import cv2_imshow
import numpy as np

##B. Siapkan Data

Seperti pada sesi sebelumnya, pada sesi ini kita juga akan menggunakan data MNIST yang sudah tersedia di librari tensorflow.keras. Dataset terdiri dari citra karakter bilangan 0 s.d. 9.

In [ ]:
#Load Dataset MNIST
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Lakukan Normalisasi
x_train, x_test = x_train / 255.0, x_test / 255.0

Kita akan bekerja dengan layer konvolusi. Pada tensorflow input untuk layer convolusi 2D harus berupa dimensi (batch, height, width, channel). Sedangkan data kita dimensinya adalah (batch, height, width), dimana batch menyatakan index data ke, dan height, width menyatakan ukuran gambar grayscale kita. Oleh karena itu, kita harus merubah dimensi tersebut dengan menggunakan fungsi reshape, sedemikian hingga dimensinya sesuai.

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train.shape[0] menyatakan batch, dalam hal ini jumlah data kita, 28,28 menyatakan ukuran masing-masing citra input kita, dan 1 menyatakan channel pada citra kita, ingat grayscale image hanya memiliki satu nilai pada setiap koordinatnya.

##C. Bangun Model

Sekarang mari buat modelnya. Kita akan membuat model CNN yang terdiri dari:
- Convolution: jml kerne: 32, ukuran kernel 3x3, fungsi aktivasi: relu
- Maxpool: ukuran kernel 2x2
- Flatten Layer: Mengubah input menjadi vektor utk input ke Fully Connected
- Hidden Layer 1: Ukuran 128 nodes, fungsi aktivasi: relu
- Output layer: Ukuran sesuai jml kelas: 10, fungsi aktivasi: softmax, 
  biasanya layer akhir klasifikasi menggunakan softmax, kenapa?

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu', input_shape=(28,28,1)),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

# Loss function yang digunakan
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

#Compile model supaya siap digunakan
model.compile(optimizer='adam',      
              loss=loss_fn,           
              metrics=['accuracy']) 

Untuk melihat bagaimana arsitektur model yang telah di-build, kita bisa menggunakna method summary().

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


##D. Lakukan Training

Setelah semua siap, mari kita lakukan training dengan memanggil method fit(). 

In [ ]:
path = "/content/drive/My Drive/Colab Notebooks/"
#Lalu buat fungsi callback
#fungsi callback untuk menyimpan model jika val_acc meningkat dari val_Acc sebelumnya
checkpoint = tf.keras.callbacks.ModelCheckpoint(path+'model-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#fungsi callback untuk menyimpan histori acc dan val-acc ke CSV
csv_logger = tf.keras.callbacks.CSVLogger(path+"log-hasil.csv", append=True, separator=';')

model.fit(x_train, y_train, 
          epochs=5,        
          callbacks=[checkpoint,csv_logger],
          validation_split=0.25)

Epoch 1/5
1404/1407 [============================>.] - ETA: 0s - loss: 0.1723 - accuracy: 0.9488
Epoch 00001: val_accuracy improved from -inf to 0.97740, saving model to /content/drive/My Drive/Colab Notebooks/model-01-0.98.hdf5
1407/1407 [==============================] - 26s 19ms/step - loss: 0.1721 - accuracy: 0.9488 - val_loss: 0.0765 - val_accuracy: 0.9774
Epoch 2/5
1406/1407 [============================>.] - ETA: 0s - loss: 0.0566 - accuracy: 0.9828
Epoch 00002: val_accuracy improved from 0.97740 to 0.97800, saving model to /content/drive/My Drive/Colab Notebooks/model-02-0.98.hdf5
1407/1407 [==============================] - 27s 19ms/step - loss: 0.0566 - accuracy: 0.9828 - val_loss: 0.0713 - val_accuracy: 0.9780
Epoch 3/5
1406/1407 [============================>.] - ETA: 0s - loss: 0.0371 - accuracy: 0.9892
Epoch 00003: val_accuracy improved from 0.97800 to 0.97993, saving model to /content/drive/My Drive/Colab Notebooks/model-03-0.98.hdf5
1407/1407 [==========================

Selesai, silakan bereksplorasi dengan parameter model lainnya.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
